# Dataset, documents, FAISS; retriever

## 🔹 Load the dataset containing the tuples `(query, correct_answer, distractor_1, distractor_2)` and the one containing the documents

In [1]:
from datasets import load_dataset

dataset = load_dataset('saracandu/filtered_hotpotQA', split="train", trust_remote_code=True)
dataset

Dataset({
    features: ['question', 'options', 'answer', 'type', 'level', 'selected_passages'],
    num_rows: 352
})

In [ ]:
# Definisci la funzione di filtro
def filter_function(example):
    return example['answer'] not in ['yes', 'no']

# Applica il filtro al dataset
filtered_dataset = dataset.filter(filter_function)
filtered_dataset

In [2]:
dataset[10]

{'question': 'Who was inducted into the Rock and Roll Hall of Fame, David Lee Roth or Cia Berg?',
 'options': "['Cia Berg', 'David Lee Roth']",
 'answer': 'David Lee Roth',
 'type': 'comparison',
 'level': 'medium',
 'selected_passages': 'Cia Berg (born 2 December 1963), now known as Cia Soro, is a Swedish television presenter and singer. She was at one time the lead singer of the Swedish rock band Whale, who released the single "Hobo Humpin\' Slobo Babe". David Lee Roth (born October 10, 1954) is an American rock vocalist, musician, songwriter, actor, author, and former radio personality. In 2007, he was inducted into the Rock and Roll Hall of Fame.'}

# Model loading and dataset selection (for testing purposes)

## ▪️ Upload the model: 

In [10]:
# do not run this unless necessary!

from huggingface_hub import login
login()

In [11]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel

#################################################################
# Tokenizer
#################################################################

model_name="nvidia/Llama3-ChatQA-1.5-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config
)

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [17]:
from guidance import models, select

new_model = models.Transformers(model, tokenizer)

new_model + f'Do you want a joke or a poem? A ' + select(['joke', 'poem'])

## ▪️ Test with `guidance`: 

In [4]:
system_message = """You are a multiple-choice question answering assistant.
Choose which of the following options: a star, a planet, a galaxy is the object below.

Object: the sun
"""

new_model + system_message + select(["a planet", "a galaxy", "a star"])

In [ ]:
from langchain import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

prompt_template = PromptTemplate.from_template(
"""You are a multiple-choice question answering assistant.
You have a suggestion on which answer is the most appropriate, that is treated as context. Use the suggestion to choose the most proper option.
You also have an attempt of answer that you are suggested to neglect. 

Question: {question}
Attempt: {candidate_answer}
Context: {critique}

The most proper option between {options} is:
"""
)

augmentation = {"question": itemgetter("question"),
                "options": itemgetter("options"), 
                "candidate_answer": itemgetter("candidate_answer"),
                "critique": itemgetter("critique"),
                "context": itemgetter("context"), }

synthesis_chain = augmentation | prompt_template 

In [ ]:
# shuffles the order of the vector containing the correct answer and the two distractors
# returns another vector, shuffled
import re
import random

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    return re.sub(r"[^\w\s.,!?\-:;()]+", '', text)

In [ ]:
def synthesisGeneration(query, prompt_template, merged, candidate_answer, critique, sources):
    # merged = ast.literal_eval(merged)
    augmented_prompt = synthesis_chain.invoke({'question': query, 
                                            'options': merged,
                                            'candidate_answer': candidate_answer,
                                            'critique': critique,
                                            'context': sources})

    normal_string = clean_text(augmented_prompt.text)
    new_model + normal_string + select(merged)

In [ ]:
synthesisGeneration('what is the sun', prompt_template, ['star', 'planet'], 'planet', 
                    'the correct answer is: a star since bot an asteroid and a planet are inadequate and not supported by the context', 
                    'The Sun is the star at the center of the Solar System. It is a massive, nearly perfect sphere of hot plasma, heated to incandescence by nuclear fusion reactions in its core, radiating the energy from its surface mainly as visible light and infrared radiation with 10% at ultraviolet energies.')

In [ ]:
source = """ Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. 
Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. 
In May 1846 it was merged into "Godey\'s Lady\'s Book. First for Women is a woman\'s magazine published by Bauer Media Group in the USA.  
The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies.
    """

synthesisGeneration('Which magazine was started first Arthur\'s Magazine or First for Women?', 
                    prompt_template, ['First for Women', 'Arthur\'s Magazine'], 'First for Women', 
                    'the correct answer is Arthur\'s Magazine and the context agrees', 
                    source)

In [ ]:
source = """ "The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group. 
The Oberoi Group is a hotel company with its head office in Delhi. 
Founded in 1934, the company owns and/or operates 30+ luxury hotels and two river cruise ships in six countries, primarily under its Oberoi Hotels & Resorts and Trident Hotels brands.".
    """

synthesisGeneration('The Oberoi family is part of a hotel company that has a head office in what city?', 
                    prompt_template, ['Delhi', 'Sammardenchia'], 'Sammardenchia', 
                    'Delhi is correct', 
                    source)

In [ ]:
source = """ Allison Beth Allie Goertz (born March 2, 1991) is an American musician. 
Goertz is known for her satirical songs based on various pop culture topics. Her videos are posted on YouTube under the name of Cossbysweater. 
Subjects of her songs have included the film The Room, the character Milhouse from the television show The Simpsons, and the game Dungeons & Dragons. 
Her style has been compared to that of Bo Burnham. In December 2015, Goertz released a concept album based on the Adult Swim series Rick and Morty, Sad Dance Songs, 
with the album\'s cover emulating the animation and logo of the series.  The album was made possible through Kickstarter. 
She is co-host of Everything's Coming Up Podcast, a Simpsons-focused podcast along with Julia Prescott. 
Milhouse Mussolini van Houten is a fictional character featured in the animated television series The Simpsons, voiced by Pamela Hayden, and created by Matt Groening 
who named the character after President Richard Nixon\'s middle name. Later in the series, it is revealed that Milhouse\'s middle name is Mussolini. "
    """

synthesisGeneration('Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?', 
                    prompt_template, ['Nixon', 'Obama'], 'Obama', 
                    'the correct answer is Nixon', 
                    source)

In [ ]:
source = """ 
Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 – June 16, 1979) was an American film director best known for the movie Rebel Without a Cause. 
Elia Kazan (born Elias Kazantzoglou September 7, 1909 – September 28, 2003) was a Greek-American director, producer, writer and actor, described by The New York Times as one of the most honored and influential directors in Broadway and Hollywood history.
    """

synthesisGeneration('What profession does Nicholas Ray and Elia Kazan have in common?', 
                    prompt_template, ['director', 'writer'], 'director', 
                    'the correct answer is writer', 
                    source)

## ▪️ Select a subset of the true dataset as a test

In [15]:
# shuffles the order of the vector containing the correct answer and the two distractors
# returns another vector, shuffled
import re
import random

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    return re.sub(r"[^\w\s.,!?\-:;()]+", '', text)

In [9]:
N_examples = 300

# select a subset of the queries, just for test:
first_queries = dataset['question'][:N_examples]

# same for correct answers and distractors:
correct_answers = dataset['answer'][:N_examples]
possibilities = dataset['options'][:N_examples]
# and for the sources:
sources = dataset['selected_passages'][:N_examples]

In [ ]:
N_examples = 237

# select a subset of the queries, just for test:
first_queries = filtered_dataset['question'][:N_examples]

# same for correct answers and distractors:
correct_answers = filtered_dataset['answer'][:N_examples]
possibilities = filtered_dataset['options'][:N_examples]
# and for the sources:
sources = filtered_dataset['selected_passages'][:N_examples]

In [7]:
sources[0]

'Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey\'s Lady\'s Book". First for Women is a woman\'s magazine published by Bauer Media Group in the USA. The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies.'

# Thesis

## 🔹 PromptTemplate definition and a LLMChain for the **thesis** 

In [4]:
# prompt template definition
# requires question, options (a string containing the possible options) and the context as input variables!

from langchain import PromptTemplate
prompt_template = PromptTemplate.from_template(
"""
    System: This is a chat between a user and an AI assistant. The assistant gives helpful, detailed, and polite answers to the user’s questions based on the context. 
    {context}
    User: {question}
    Possible options: {options}.
    Assistant:
"""
)

## 🔹 Function that generates the output given the prompt, the question and the set of options

In [5]:
# LLM chain definition
from operator import itemgetter

augmentation = {"question": itemgetter("question"),
                "options": itemgetter("options"),
                "context": itemgetter("context"), }

thesis_chain = augmentation | prompt_template 

In [13]:
import ast

def thesisGeneration(query, merged, sources):
    merged = ast.literal_eval(merged)
    augmented_prompt = thesis_chain.invoke({'question': query, 'options': merged, 'context': sources})
    normal_string = clean_text(augmented_prompt.text)
    ans = new_model + normal_string + select([clean_text(merged[0]), clean_text(merged[1])])
    return str(ans)

In [7]:
def extract_answer(text):
    # Trova l'indice in cui inizia il testo "Why or why not the answer is correct:"
    start_index = text.find("Assistant:")

    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        start_index += len("Assistant:")
        # Estrai il testo dopo "Why or why not the answer is correct:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

## 🔹 Test: how well the thesis alone is able to perform?

In [18]:
answers = []
for i in range(N_examples):
    answers.append(extract_answer(thesisGeneration(first_queries[i], possibilities[i], sources[i])))

In [19]:
answers[:5]

['First for Women', 'Jonathan Stark', 'The Wolfhounds', 'yes', 'yes']

In [ ]:
correct_answers[:5]

# Antithesis

In [ ]:
import transformers

pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=False,
    temperature=0.0,
    repetition_penalty=1.5,
    return_full_text=True,
    max_new_tokens=400,
    top_p=0.0
)

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown

def query_model(
        system_message,
        user_message,
        temperature = 0.0,
        max_length=1024
        ):

    user_message = "Question: " + user_message + " Correct answer:"
    messages = [
        {"role": "System", "content": system_message},
        {"role": "User", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        do_sample=False,
        top_p=0.0,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )

    answer = sequences[0]['generated_text']

    return answer  

In [16]:
system_message = """
    This is a chat between a user and an AI assistant.
    The assistant is asked to determine the most correct answer for a given question, provided a set of possible options.
    It also has a first tentative answer that is required to check with respect to the question and the relevant context:
    {context}
    The goal of the assistant is to decree which is the most correct answer to the question between the available options.
    It can consider the attempt of answer and check whether or not it is correct given the question and the context.
"""


user_message = """
    Question: {question}?
    It also has an attempt of answer: {candidate_answer}
    Which of the candidate answers {options} is the most proper answer for the question? Why? 
    Think step by step but choose only one of the options: {options} by explicitly reporting which one.
"""

## 🔸 Function to generate the antithesis given the question, the thesis, the context and the options

In [46]:
def antithesisGeneration(query, prompt_template, merged, candidate_answer, sources):
    merged = ast.literal_eval(merged)
    second_answer = query_model(system_message.format(context = sources),
    user_message.format(question=query, options = merged, candidate_answer = candidate_answer, context = sources,), max_length=1024)
    return second_answer

In [49]:
ant_answers = []
for i in range(10):
    ant_answers.append(antithesisGeneration(first_queries[i], prompt_template, possibilities[i], answers[i], sources[i]))

In [50]:
ant_answers[:10]

['The reference text states,"Edited by TS arthur,it feautred works b y edgar p oe"',
 'The appropriate response would have been "Leconté" because this individual had achieved greater success than any other competitor throughout their careers as professionals within respective fields; whereas stark did manage winning some major tournaments yet never managed reaching heights comparable too lecon',
 "'Hole'",
 'The two men were well-known scientists from different fields; therefore their names would have been familiar even if they had never met each other before becoming friends later down life path as adults after high school graduation ceremony where all graduates receive diploma certificate proving successful completion degree program requirements towards earning bachelor’s/master/doctorate degrees depending upon individual choice made earlier during application process while still minors under legal guardianship until adulthood age limit reached at eighteen years old',
 'The reference

In [ ]:
first_queries[:10]

In [ ]:
correct_answers[:5]

# Synthesis

## 🔺 PromptTemplate definition and a LLMChain for the **synthesis** 

In [21]:
prompt_template = PromptTemplate.from_template(
"""You are a multiple-choice question answering assistant.
You have a suggestion on which answer is the most appropriate that you are strongly suggested to follow.
Choose the most proper option between {options} that best matches with the suggestion. You also have sources reported in case you need them. 

Question: {question}
Context: The proposed answer is {candidate_answer} and another suggested that {critique}
Sources: {context}

Assistant:
"""
)

augmentation = {"question": itemgetter("question"),
                "options": itemgetter("options"), 
                "candidate_answer": itemgetter("candidate_answer"),
                "critique": itemgetter("critique"),
                "context": itemgetter("context"), }

synthesis_chain = augmentation | prompt_template 

## 🔺 Function to generate the synthesis given literally everything

In [22]:
def synthesisGeneration(query, prompt_template, merged, candidate_answer, critique, sources):
    merged = ast.literal_eval(merged)
    augmented_prompt = synthesis_chain.invoke({'question': query, 
                                            'options': merged,
                                            'candidate_answer': candidate_answer,
                                            'critique': critique,
                                            'context': sources})

    normal_string = clean_text(augmented_prompt.text)
    ans = new_model + normal_string + select([clean_text(merged[0]), clean_text(merged[1])])
    return str(ans)

In [ ]:
def_answers = ["the correct option is " + clean_text(correct_answer)
               + "since the other options is not mentioned in the context" for correct_answer in correct_answers]

In [23]:
syn_answers = []
for i in range(N_examples):
    syn_answers.append(extract_answer(
        synthesisGeneration(
            first_queries[i], prompt_template, possibilities[i], answers[i], 
            ant_answers[i], sources[i])))

In [24]:
syn_answers[:10]

['Arthurs Magazine',
 'Jonathan Stark',
 'The Wolfhounds',
 'yes',
 'yes',
 'New York City',
 'Aleksander Ford',
 'yes',
 'director',
 'The Saimaa Gesture']

In [ ]:
correct_answers[:10]

In [ ]:
answers[:10]

In [25]:
df = {
    'query': first_queries,
    'correct': correct_answers,
    'thesis': answers,
    'antithesis': ant_answers,
    'synthesis': syn_answers,
    'context': sources
} 

In [27]:
import pandas as pd

df = pd.DataFrame(df)
df.head()

,query,correct,thesis,antithesis,synthesis,context
0,Which magazine was started first Arthur's Maga...,Arthur's Magazine,First for Women,"The reference text states,""Edited by TS arthur...",Arthurs Magazine,Arthur's Magazine (1844–1846) was an American ...
1,Which tennis player won more Grand Slam titles...,Jonathan Stark,Jonathan Stark,"The appropriate response would have been ""Leco...",Jonathan Stark,Henri Leconte (born 4 July 1963) is a former F...
2,"Which band was founded first, Hole, the rock b...",The Wolfhounds,The Wolfhounds,'Hole',The Wolfhounds,The Wolfhounds are an indie pop/noise pop band...
3,Were Pavel Urysohn and Leonid Levin known for ...,no,yes,The two men were well-known scientists from di...,yes,Leonid Anatolievich Levin ( ; Russian: Леони́д...
4,Are both The New Pornographers and Kings of Le...,no,yes,"The reference text states ""Kings Of Leon are A...",yes,Kings of Leon is an American rock band that fo...


In [ ]:
df['query'][3]

In [44]:
df.to_csv('baseline.csv')

In [28]:
# Funzione per rimuovere le quadre e ottenere solo il contenuto
def remove_brackets(s):
    return s.strip("[] ")

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    text = re.sub(r'[^\w\s.,!?\'"\-:;()]+', '', text)  # Rimuove i caratteri speciali
    text = re.sub(r"['\"-]", '', text)  # Rimuove apostrofi, virgolette e trattini
    text = text.lower()  # Converte in minuscolo
    return text

# Applica la funzione alla colonna 'correct answer'
df['correct'] = df['correct'].apply(clean_text)
df['thesis'] = df['thesis'].apply(clean_text)
df['synthesis'] = df['synthesis'].apply(clean_text)

df.head()

,query,correct,thesis,antithesis,synthesis,context
0,Which magazine was started first Arthur's Maga...,arthurs magazine,first for women,"The reference text states,""Edited by TS arthur...",arthurs magazine,Arthur's Magazine (1844–1846) was an American ...
1,Which tennis player won more Grand Slam titles...,jonathan stark,jonathan stark,"The appropriate response would have been ""Leco...",jonathan stark,Henri Leconte (born 4 July 1963) is a former F...
2,"Which band was founded first, Hole, the rock b...",the wolfhounds,the wolfhounds,'Hole',the wolfhounds,The Wolfhounds are an indie pop/noise pop band...
3,Were Pavel Urysohn and Leonid Levin known for ...,no,yes,The two men were well-known scientists from di...,yes,Leonid Anatolievich Levin ( ; Russian: Леони́д...
4,Are both The New Pornographers and Kings of Le...,no,yes,"The reference text states ""Kings Of Leon are A...",yes,Kings of Leon is an American rock band that fo...


In [52]:
df['context'][0]

'Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey\'s Lady\'s Book". First for Women is a woman\'s magazine published by Bauer Media Group in the USA. The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies.'

In [37]:
# Conta quante righe combaciano e quante no
matches = 0
non_matches = 0
which_ones = []

for index, row in df.iterrows():
    correct_answer = str(row['correct']).strip()
    thesis = str(row['thesis']).strip()
    
    if correct_answer == thesis:
        matches += 1
    else:
        non_matches += 1
        which_ones.append("thesis: {}, Correct: {}".format(thesis, correct_answer))

print(f"Number of matches: {matches}")
print(f"Number of non-matches: {non_matches}")

Number of matches: 218
Number of non-matches: 82


In [38]:
which_ones

['thesis: first for women, Correct: arthurs magazine',
 'thesis: yes, Correct: no',
 'thesis: yes, Correct: no',
 'thesis: yes, Correct: no',
 'thesis: ross lynch, Correct: glenn hughes',
 'thesis: kim clijsters, Correct: mary pierce',
 'thesis: edwyn collins, Correct: jimmie ross',
 'thesis: the hunchback of notre dame, Correct: saludos amigos',
 'thesis: temagamilorrain mine, Correct: meadowbank gold mine',
 'thesis: neil labute, Correct: h. bruce humberstone',
 'thesis: yes, Correct: no',
 'thesis: jack kevorkian, Correct: jacob kevorkian',
 'thesis: luis muñoz marín international airport, Correct: rickenbacker international airport',
 'thesis: crime novelist, Correct: playwright',
 'thesis: xavier malisse, Correct: virginia wade',
 'thesis: george marshall, Correct: allan dwan',
 'thesis: yes, Correct: no',
 'thesis: cardwellia, Correct: ochagavia',
 'thesis: joe manganiello, Correct: kerkor kerkorian',
 'thesis: chinese crested dog, Correct: chiengris',
 'thesis: vincent walker, C

In [40]:
# Conta quante righe combaciano e quante no
matches = 0
non_matches = 0
which_ones_syn = []

for index, row in df.iterrows():
    correct_answer = str(row['correct']).strip()
    thesis = str(row['synthesis']).strip()
    
    if correct_answer == thesis:
        matches += 1
    else:
        # print("Synthesis: {}, Correct: {}".format(thesis, correct_answer))
        non_matches += 1
        which_ones_syn.append("Synthesis: {}, Correct: {}".format(thesis, correct_answer))

print(f"Number of matches: {matches}")
print(f"Number of non-matches: {non_matches}")

Number of matches: 220
Number of non-matches: 80


In [41]:
which_ones_syn

['Synthesis: yes, Correct: no',
 'Synthesis: yes, Correct: no',
 'Synthesis: yes, Correct: no',
 'Synthesis: ross lynch, Correct: glenn hughes',
 'Synthesis: producer, Correct: filmmaker',
 'Synthesis: no, Correct: yes',
 'Synthesis: kim clijsters, Correct: mary pierce',
 'Synthesis: edwyn collins, Correct: jimmie ross',
 'Synthesis: robert e. howard, Correct: richard ford',
 'Synthesis: the hunchback of notre dame, Correct: saludos amigos',
 'Synthesis: temagamilorrain mine, Correct: meadowbank gold mine',
 'Synthesis: jack kevorkian, Correct: jacob kevorkian',
 'Synthesis: luis muñoz marín international airport, Correct: rickenbacker international airport',
 'Synthesis: crime novelist, Correct: playwright',
 'Synthesis: xavier malisse, Correct: virginia wade',
 'Synthesis: george marshall, Correct: allan dwan',
 'Synthesis: yes, Correct: no',
 'Synthesis: cardwellia, Correct: ochagavia',
 'Synthesis: no, Correct: yes',
 'Synthesis: joe manganiello, Correct: kerkor kerkorian',
 'Synth

In [42]:
220/(220+80)*100

73.33333333333333

In [43]:
222/(222+78)*100

74.0